# Utilities to test DynamoDb and S3 database for TEMCA Optical images

In [15]:
import boto3

# keep the following consistent per run
specimen_id = 'donaldtrump'
media_id = 'test1'
s3bucket = 'temcatester'
region = 'us-west-2'

In [16]:
dynamodb = boto3.resource('dynamodb', region_name=region)

In [14]:
# Create the TAO table.  Just do this once.
table = dynamodb.create_table(
    TableName='TAO',
    KeySchema=[
        {
            'AttributeName': 'specimenId_mediaId',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'grid',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'specimenId_mediaId',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'grid',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

print("Table status:", table.table_status)

('Table status:', u'CREATING')


In [ ]:
# Create the S3 bucket
s3 = boto3.resource('s3')
s3.create_bucket(Bucket=s3bucket, CreateBucketConfiguration={'LocationConstraint': region})

In [42]:
# Get status of DynamoDB and S3
table = dynamodb.Table('TAO')
print("Table status:", table.table_status)
print("Item count: ", table.item_count)

response = table.get_item( Key = {
        'specimenId_mediaId': specimen_id + "_" + media_id,
        'grid': "000051"
    })
item = response['Item']
print(item)

('Table status:', u'ACTIVE')
('Item count: ', 0)
{u'bucket': u'temcatester', u'aperture_im': u'160721_111223ximeatest.jpeg', u'tao': {u'specimen_id': u'donaldtrump', u'media_id': u'test1', u'grids': {u'000051': {u'rois': []}}, u'media': u'tape'}, u'grid': u'000051', u'specimenId_mediaId': u'donaldtrump_test1'}


In [43]:
# push images and TAOs into the DB
import os
from subprocess import Popen, PIPE

rootdir = r"\\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages"
count = 0

bucket = s3.Bucket(s3bucket)
table = dynamodb.Table('TAO')

for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith(".jpeg"):
            fpath = os.path.join(root, file) 
            p = Popen(['zbarimg.exe', '--raw', fpath], stdout=PIPE, stderr=PIPE, stdin=PIPE)
            barcode = p.stdout.read().strip() 
            
            print count, barcode, fpath
            count = count + 1
            #print len(barcode)
            if (len(barcode) != 6):
                print ('barcode decode failed for: ' + fpath)
                continue
            else:
                tao = {
                    'specimen_id' : specimen_id,
                    'media': 'tape',
                    'media_id': media_id,
                    'grids': {
                        barcode : {
                            'rois': []
                        }
                    }
                }

                item = {
                    'specimenId_mediaId': specimen_id + "_" + media_id,
                    'grid' : barcode,
                    'tao': tao,
                    'bucket': s3bucket,
                    'aperture_im': file
                }
                    
            try:
                print (item)
                response = table.put_item(Item = item)
                print (response)
            except Exception as e:
                print('DynamoDb put_item failed: ' + str(e))
                
            try:
                response = s3.Bucket(s3bucket).upload_file(fpath, file)
                print (response)
            except  Exception as e:
                print('S3 upload failed: ' + e + file)
            #if (count > 3):
            #    break



0 000051 \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111222ximeatest.jpeg
{'bucket': 'temcatester', 'aperture_im': '160721_111222ximeatest.jpeg', 'tao': {'specimen_id': 'donaldtrump', 'media_id': 'test1', 'grids': {'000051': {'rois': []}}, 'media': 'tape'}, 'grid': '000051', 'specimenId_mediaId': 'donaldtrump_test1'}
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '8DMIETEC206L03S1NU3L7SGV2JVV4KQNSO5AEMVJF66Q9ASUAAJG'}}
None
1 000051 \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111223ximeatest.jpeg
{'bucket': 'temcatester', 'aperture_im': '160721_111223ximeatest.jpeg', 'tao': {'specimen_id': 'donaldtrump', 'media_id': 'test1', 'grids': {'000051': {'rois': []}}, 'media': 'tape'}, 'grid': '000051', 'specimenId_mediaId': 'donaldtrump_test1'}
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'DDKKMD16NSUJ71SMVQKIT2O3AVVV4KQNSO5AEMVJF66Q9ASUAAJG'}}
None
2  \\AIBSDATA2\nc-em\adamb\atumDev